In [1]:
import tensorflow as tf
import pickle
import numpy as np
import timeit
import load_cifar
from sklearn.utils import shuffle
from tensorflow.contrib.layers import flatten

In [2]:
tf.reset_default_graph()

In [3]:
val_features, val_labels = pickle.load(open('val_data.pickle','rb'))
train_features,train_labels = pickle.load(open('train_data.pickle','rb'))
test_features, test_labels = pickle.load(open('test_data.pickle','rb'))
train_features = load_cifar.features_reshape(train_features)
val_features = load_cifar.features_reshape(val_features)
test_features = load_cifar.features_reshape(test_features)
val_labels = np.squeeze(val_labels.todense())
train_labels = np.squeeze(train_labels.todense())
test_labels = np.squeeze(test_labels.todense())

<h1>Hyperparameter</h1>

In [4]:
#learning rate
lr = 0.0008
#number of traning epochs
epochs = 10
#number of batch_size
batch_size = 32
total_batch = int(40000/batch_size)
num_steps = (epochs+1) * total_batch
num_classes = 10
W = 32
H = 32
channel = 3

<h1>Placeholder</h1>

In [5]:
X = tf.placeholder(tf.float32,[None,W, H, channel],name='X')
Y = tf.placeholder(tf.int32,[None,num_classes],name='Y')

<h1>LeNet-5</h1>

In [6]:
def LeNet(x):    
    # Conv Layer 1
    conv1_w = tf.Variable(tf.truncated_normal(shape = [5,5,3,6], mean = 0, stddev = 0.1))
    conv1_b = tf.Variable(tf.zeros(6))
    conv1 = tf.nn.conv2d(x,conv1_w, strides = [1,1,1,1], padding = 'VALID') + conv1_b
    conv1 = tf.nn.relu(conv1)

    # Pooling Layer 1
    pool_1 = tf.nn.max_pool(conv1, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID')
    
    # Conv Layer 2
    conv2_w = tf.Variable(tf.truncated_normal(shape = [5,5,6,16], mean = 0, stddev = 0.1))
    conv2_b = tf.Variable(tf.zeros(16))
    conv2 = tf.nn.conv2d(pool_1, conv2_w, strides = [1,1,1,1], padding = 'VALID') + conv2_b
    conv2 = tf.nn.relu(conv2)

    # Pooling Layer 2
    pool_2 = tf.nn.max_pool(conv2, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID') 
    
    # Conv Layer 3
    conv3 = flatten(pool_2)
    conv3_w = tf.Variable(tf.truncated_normal(shape = (400,120), mean = 0, stddev = 0.1))
    conv3_b = tf.Variable(tf.zeros(120))
    conv3 = tf.matmul(conv3,conv3_w) + conv3_b
    conv3 = tf.nn.relu(conv3)

    # Fully Connected Layer 1
    fc2_w = tf.Variable(tf.truncated_normal(shape = (120,84), mean = 0, stddev = 0.1))
    fc2_b = tf.Variable(tf.zeros(84))
    fc2 = tf.matmul(conv3,fc2_w) + fc2_b
    fc2 = tf.nn.relu(fc2)
    
    # Fully Connected Layer 2
    fc3_w = tf.Variable(tf.truncated_normal(shape = (84,10), mean = 0, stddev = 0.1))
    fc3_b = tf.Variable(tf.zeros(10))
    logits = tf.matmul(fc2, fc3_w) + fc3_b
    return logits

<h1>Cost and Optimization</h1>

In [7]:
logits = LeNet(X)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate = lr)
train_op = optimizer.minimize(cost)

correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

<h1>Training, validation and testing</h1>
<h2>Train your model only 10 epochs.</h2>
<h2>1.Print out validation accuracy after each training epoch</h2>
<h2>2.Print out training time for each training epoch</h2>
<h2>3.Print out testing accuracy</h2>

In [8]:
def next_batch(batch_size):
    idx = np.arange(0 , len(train_features))
    np.random.shuffle(idx)
    idx = idx[:batch_size]
    data_shuffle = [train_features[i] for i in idx]
    labels_shuffle = [train_labels[i] for i in idx]
    data_shuffle = np.asarray(data_shuffle)
    labels_shuffle = np.asarray(labels_shuffle)
    labels_shuffle = np.squeeze(labels_shuffle)
    return data_shuffle, labels_shuffle

In [9]:
with tf.Session() as sess:
    sess.run(init)
    # Timer
    for i in range(num_steps):
        # fetch batch
        batch_x, batch_y = next_batch(batch_size)
        # run optimization
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if (i > 1) and (i % total_batch == 0):
            acc = sess.run(accuracy, feed_dict={X: batch_x, Y: batch_y})
            print("Epoch " + str(i/total_batch) + ", Accuracy: {:.3f}".format(acc))

    print("Training finished!")

    print("Testing Accuracy::{:.3f}".format(float(sess.run(accuracy, feed_dict={X: test_features, Y: test_labels}))))

Epoch 1.0, Accuracy: 0.594
Epoch 2.0, Accuracy: 0.312
Epoch 3.0, Accuracy: 0.781
Epoch 4.0, Accuracy: 0.594
Epoch 5.0, Accuracy: 0.812
Epoch 6.0, Accuracy: 0.688
Epoch 7.0, Accuracy: 0.594
Epoch 8.0, Accuracy: 0.844
Epoch 9.0, Accuracy: 0.594
Epoch 10.0, Accuracy: 0.594
Training finished!
Testing Accuracy::0.599
